In [1]:
import cv2
import os
import numpy as np

In [26]:
def resize_to_half(image_path):
    """Resize the image to half of its original dimensions."""
    image = cv2.imread(image_path)
    half_size = (image.shape[1] // 2, image.shape[0] // 2)
    resized_image = cv2.resize(image, half_size, interpolation=cv2.INTER_AREA)
    return resized_image

In [27]:
def crop_center(image, cropx=256, cropy=256):
    """Crop the 256x256 center of the image."""
    y, x = image.shape[0:2]
    startx = x // 2 - (cropx // 2)
    starty = y // 2 - (cropy // 2)    
    return image[starty:starty+cropy, startx:startx+cropx]

In [54]:
def convert_to_grayscale(image):
    """Convert the image to grayscale."""
    return cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

In [55]:
def process_images(folder_path, output_folder_path):
    """Process all images in the given folder, renaming and saving those that are exactly 256x256 after processing."""
    counter = 0  # Initialize counter for naming files sequentially
    for filename in os.listdir(folder_path):
        # Check if the file is an image (simple check based on file extension)
        if filename.lower().endswith(('.png', '.jpg', '.jpeg')):
            image_path = os.path.join(folder_path, filename)
            
            # Step 1: Resize the image to half of its original dimensions
            resized_image = resize_to_half(image_path)
            
            # Step 2: Crop the 256x256 center of the resized image
            cropped_image = crop_center(resized_image)
            
            # Check if the cropped image is exactly 256x256
            if cropped_image.shape[:2] == (256, 256):
                # Step 3: Convert the cropped image to grayscale
                grayscale_image = convert_to_grayscale(cropped_image)
                
                # Construct the new filename based on the counter
                new_filename = f"{counter:03d}.png"  # Assuming you want to save as PNG
                output_path = os.path.join(output_folder_path, new_filename)
                
                # Save the final grayscale image to the output folder
                cv2.imwrite(output_path, grayscale_image)
                print(f"Processed and saved: {output_path}")
                
                counter += 1  # Increment counter only when an image is successfully processed and saved
            else:
                print(f"Skipped {filename}: Cropped image size is not 256x256.")

In [56]:
folder_path = '../../data/div2k-lowres'
output_folder_path = '../../data/div2k-denoise/clean'

In [57]:
process_images(folder_path, output_folder_path)

Processed and saved: ../../data/div2k-denoise/clean\000.png
Processed and saved: ../../data/div2k-denoise/clean\001.png
Processed and saved: ../../data/div2k-denoise/clean\002.png
Processed and saved: ../../data/div2k-denoise/clean\003.png
Processed and saved: ../../data/div2k-denoise/clean\004.png
Processed and saved: ../../data/div2k-denoise/clean\005.png
Processed and saved: ../../data/div2k-denoise/clean\006.png
Processed and saved: ../../data/div2k-denoise/clean\007.png
Processed and saved: ../../data/div2k-denoise/clean\008.png
Processed and saved: ../../data/div2k-denoise/clean\009.png
Processed and saved: ../../data/div2k-denoise/clean\010.png
Processed and saved: ../../data/div2k-denoise/clean\011.png
Processed and saved: ../../data/div2k-denoise/clean\012.png
Processed and saved: ../../data/div2k-denoise/clean\013.png
Processed and saved: ../../data/div2k-denoise/clean\014.png
Processed and saved: ../../data/div2k-denoise/clean\015.png
Processed and saved: ../../data/div2k-de

In [2]:
def add_gaussian_noise(image, mean=0, sigma=25):
    row, col = image.shape
    gaussian = np.random.normal(mean, sigma, (row, col))
    noisy_image = np.clip(image + gaussian, 0, 255).astype(np.uint8)
    return noisy_image

In [3]:
def add_salt_pepper_noise(image, salt_prob=0.01, pepper_prob=0.01):
    row, col = image.shape
    noisy_image = np.copy(image)
    # Salt noise
    num_salt = np.ceil(salt_prob * image.size)
    coords = [np.random.randint(0, i - 1, int(num_salt)) for i in [row, col]]
    noisy_image[coords] = 255
    # Pepper noise
    num_pepper = np.ceil(pepper_prob * image.size)
    coords = [np.random.randint(0, i - 1, int(num_pepper)) for i in [row, col]]
    noisy_image[coords] = 0
    return noisy_image

In [39]:
def add_poisson_noise(image):
    vals = len(np.unique(image))
    vals = 2 ** np.ceil(np.log2(vals))
    noisy_image = np.random.poisson(image * vals) / float(vals)
    return noisy_image.astype(np.uint8)

In [44]:
def add_speckle_noise(image, sigma = 0.5):

    mean = 1.0  # Mean of the Gaussian distribution
    noise = np.random.normal(mean, sigma, image.shape)

    # Add speckle noise to the image
    noisy_image = image + image * noise

    # Convert back to uint8 format
    noisy_image_uint8 = np.clip(noisy_image, 0, 255).astype(np.uint8)

    return noisy_image_uint8

In [45]:
def generate_noisy_images(folder_path, output_base_path):
    """Process all images in the given folder, and apply different noise types to processed images."""
    count = 0  # Initialize file name counter
    # Create directories for each noise type
    os.makedirs(os.path.join(output_base_path, 'gaussian'), exist_ok=True)
    os.makedirs(os.path.join(output_base_path, 'salt-pepper'), exist_ok=True)
    os.makedirs(os.path.join(output_base_path, 'poisson'), exist_ok=True)
    os.makedirs(os.path.join(output_base_path, 'speckle'), exist_ok=True)

    for filename in sorted(os.listdir(folder_path)):
        # Check if the file is an image
        if filename.lower().endswith(('.png', '.jpg', '.jpeg')):
            image_path = os.path.join(folder_path, filename)
            image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
            
            # Apply noise and save in respective folders
            gaussian_image = add_gaussian_noise(image)
            cv2.imwrite(os.path.join(output_base_path, 'gaussian', f'{count:03d}.png'), gaussian_image)
            
            sp_image = add_salt_pepper_noise(image)
            cv2.imwrite(os.path.join(output_base_path, 'salt-pepper', f'{count:03d}.png'), sp_image)
            
            poisson_image = add_poisson_noise(image)
            cv2.imwrite(os.path.join(output_base_path, 'poisson', f'{count:03d}.png'), poisson_image)
            
            speckle_image = add_speckle_noise(image)
            cv2.imwrite(os.path.join(output_base_path, 'speckle', f'{count:03d}.png'), speckle_image)
            
            print(f"Processed and saved image set: {count:03d}")
            count += 1  # Increment counter after successfully processing and saving an image set

In [46]:
clean_img_path = '../../data/div2k-denoise/clean'
output_base_path = '../../data/div2k-denoise'

In [47]:
generate_noisy_images(clean_img_path,output_base_path)

C:\Users\wishm\anaconda3\envs\msc-env\lib\site-packages\ipykernel_launcher.py:7: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  import sys
C:\Users\wishm\anaconda3\envs\msc-env\lib\site-packages\ipykernel_launcher.py:11: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  # This is added back by InteractiveShellApp.init_path()


Processed and saved image set: 000
Processed and saved image set: 001
Processed and saved image set: 002
Processed and saved image set: 003
Processed and saved image set: 004
Processed and saved image set: 005
Processed and saved image set: 006
Processed and saved image set: 007
Processed and saved image set: 008
Processed and saved image set: 009
Processed and saved image set: 010
Processed and saved image set: 011
Processed and saved image set: 012
Processed and saved image set: 013
Processed and saved image set: 014
Processed and saved image set: 015
Processed and saved image set: 016
Processed and saved image set: 017
Processed and saved image set: 018
Processed and saved image set: 019
Processed and saved image set: 020
Processed and saved image set: 021
Processed and saved image set: 022
Processed and saved image set: 023
Processed and saved image set: 024
Processed and saved image set: 025
Processed and saved image set: 026
Processed and saved image set: 027
Processed and saved 